# Neural Language Model
A language model predicts the next word in the sequence based on the specific words that have come before it in the sequence.

It is also possible to develop language models at the character level using neural networks. The benefit of character-based language models is their small vocabulary and flexibility in handling any words, punctuation, and other document structure. This comes at the cost of requiring larger models that are slower to train.

Nevertheless, in the field of neural language models, character-based models offer a lot of promise for a general, flexible and powerful approach to language modeling.

As a prerequisite for the lab, make sure to pip install:
- keras
- tensorflow
- h5py

# Source Text Creation

To start out with, we'll be using a simple nursery rhyme. It's quite short so we can actually train something on your CPU and see relatively interesting results. Please copy and paste the following text in a text file and save it as "rhyme.txt". Place this in the same directory as this jupyter notebook:

In [ ]:
!pip install tensorflow
!pip install keras
!pip install h5py

In [ ]:
from keras.layers import Dropout

Using TensorFlow backend.


In [ ]:
s='Sing a song of sixpence,\
A pocket full of rye.\
Four and twenty blackbirds,\
Baked in a pie.\
When the pie was opened\
The birds began to sing;\
Wasn’t that a dainty dish,\
To set before the king.\
The king was in his counting house,\
Counting out his money;\
The queen was in the parlour,\
Eating bread and honey.\
The maid was in the garden,\
Hanging out the clothes,\
When down came a blackbird\
And pecked off her nose.'

with open('rhymes.txt','w') as f:
  f.write(s)

    Sing a song of sixpence,
    A pocket full of rye.
    Four and twenty blackbirds,
    Baked in a pie.

    When the pie was opened
    The birds began to sing;
    Wasn’t that a dainty dish,
    To set before the king.

    The king was in his counting house,
    Counting out his money;
    The queen was in the parlour,
    Eating bread and honey.

    The maid was in the garden,
    Hanging out the clothes,
    When down came a blackbird
    And pecked off her nose.

# Sequence Generation

A language model must be trained on the text, and in the case of a character-based language model, the input and output sequences must be characters.

The number of characters used as input will also define the number of characters that will need to be provided to the model in order to elicit the first predicted character.

After the first character has been generated, it can be appended to the input sequence and used as input for the model to generate the next character.

Longer sequences offer more context for the model to learn what character to output next but take longer to train and impose more burden on seeding the model when generating text.

We will use an arbitrary length of 10 characters for this model.

There is not a lot of text, and 10 characters is a few words.

We can now transform the raw text into a form that our model can learn; specifically, input and output sequences of characters.

In [ ]:
#load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [ ]:
#load text
raw_text = load_doc('rhymes.txt')
print(raw_text)

# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

Sing a song of sixpence,A pocket full of rye.Four and twenty blackbirds,Baked in a pie.When the pie was openedThe birds began to sing;Wasn’t that a dainty dish,To set before the king.The king was in his counting house,Counting out his money;The queen was in the parlour,Eating bread and honey.The maid was in the garden,Hanging out the clothes,When down came a blackbirdAnd pecked off her nose.
Total Sequences: 384


In [ ]:
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

# Train a Model
In this section, we will develop a neural language model for the prepared sequence data.

The model will read encoded characters and predict the next character in the sequence. A Long Short-Term Memory recurrent neural network hidden layer will be used to learn the context from the input sequence in order to make the predictions.

In [ ]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [ ]:
# load

in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

The sequences of characters must be encoded as integers.This means that each unique character will be assigned a specific integer value and each sequence of characters will be encoded as a sequence of integers. We can create the mapping given a sorted set of unique characters in the raw input data. The mapping is a dictionary of character values to integer values.

Next, we can process each sequence of characters one at a time and use the dictionary mapping to look up the integer value for each character. The result is a list of integer lists.

We need to know the size of the vocabulary later. We can retrieve this as the size of the dictionary mapping.

In [ ]:
# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append(encoded_seq)

# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 38


The model is defined with an input layer that takes sequences that have 10 time steps and 38 features for the one hot encoded input sequences. Rather than specify these numbers, we use the second and third dimensions on the X input data. This is so that if we change the length of the sequences or size of the vocabulary, we do not need to change the model definition.

The model has a single LSTM hidden layer with 75 memory cells. The model has a fully connected output layer that outputs one vector with a probability distribution across all characters in the vocabulary. A softmax activation function is used on the output layer to ensure the output has the properties of a probability distribution.

The model is learning a multi-class classification problem, therefore we use the categorical log loss intended for this type of problem. The efficient Adam implementation of gradient descent is used to optimize the model and accuracy is reported at the end of each batch update. The model is fit for 50 training epochs.

# To Do:
- Try different numbers of memory cells
- Try different types and amounts of recurrent and fully connected layers
- Try different lengths of training epochs
- Try different sequence lengths and pre-processing of data
- Try regularization techniques such as Dropout

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# define model
model = Sequential()
model.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history=model.fit(X, y, validation_split = 0.3, epochs=80)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 200)               191200    
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 38)                7638      
Total params: 198,838
Trainable params: 198,838
Non-trainable params: 0
_________________________________________________________________
None
Train on 268 samples, validate on 116 samples
Epoch 1/80
268/268 [==============================] - 2s 8ms/step - loss: 3.6143 - acc: 0.0821 - val_loss: 3.5595 - val_acc: 0.1552
Epoch 2/80
268/268 [==============================] - 0s 1ms/step - loss: 3.4493 - acc: 0.1604 - val_loss: 3.2009 - val_acc: 0.1552
Epoch 3/80
268/268 [==============================] - 0s 1ms/st

In [ ]:
results = model.evaluate(X_test, y_test)
print(results)

77/77 [==============================] - 0s 312us/step
[1.430651663185714, 0.7922077937559648]


In [ ]:
# save the model to file
model.save('model.h5')
# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

# Generating Text

We must provide sequences of 10 characters as input to the model in order to start the generation process. We will pick these manually. A given input sequence will need to be prepared in the same way as preparing the training data for the model. 

In [ ]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        #encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

Running the example generates three sequences of text.

The first is a test to see how the model does at starting from the beginning of the rhyme. The second is a test to see how well it does at beginning in the middle of a line. The final example is a test to see how well it does with a sequence of characters never seen before.

In [ ]:
# test start of rhyme
print(generate_seq(model, mapping, 10, 'Sing a son', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'hello worl', 20))

Sing a song of sixpence,A pock
king was in his counting house
hello worl..FFurttnradddaaidtt


If the results aren't satisfactory, try out the suggestions above or these below:
- Padding. Update the example to provides sequences line by line only and use padding to fill out each sequence to the maximum line length.
- Sequence Length. Experiment with different sequence lengths and see how they impact the behavior of the model.
- Tune Model. Experiment with different model configurations, such as the number of memory cells and epochs, and try to develop a better model for fewer resources.


# Deliverables to receive credit

1. (3 points) Optimize the cells above to get the model to work reasonably well on the generated sentences. Again, this is a toy problem as language models require a lot of computation... so this toy problem is great for rapid experimentation to explore different aspects of deep learning language models.
2. (2 points) Write a function to split the text corpus file into training and validation and pipe the validation data into the model.fit() function to be able to track validation error per epoch. Lookup Keras documentation to see how this is handled.
3. (2 points) Write a summary (methods and results) in the cells below of the different things you applied. You must include your intuitions behind what did work and what did not work well.
4. (3 points) Do something even more interesting. Try a different source text. Train a word-level model. We'll leave it up to your creativity to explore and write a summary of your methods and results.


In [ ]:
"summary below"

'summary below'

I tried different combinations of hidden layers (Depth) and epochs and using Dropout at different locations. For some reason, adding another layer makes the accuracy of the test incredibly low, so I did not include it. Increasing the epochs allowed the model to train more, making the prediction better. Having it at 100, however, felt like it was too much and overtraining. Including the Dropout was in hopes of decreasing that effect.

In [ ]:
a="To be, or not to be, that is the question:\
Whether tis nobler in the mind to suffer\
The slings and arrows of outrageous fortune,\
Or to take arms against a sea of troubles\
And by opposing end them. To die—to sleep,\
No more; and by a sleep to say we end\
The heart-ache and the thousand natural shocks\
That flesh is heir to: 'tis a consummation\
Devoutly to be wish'd. To die, to sleep;\
To sleep, perchance to dream—ay, there's the rub:\
For in that sleep of death what dreams may come,\
When we have shuffled off this mortal coil,\
Must give us pause—there's the respect\
That makes calamity of so long life.\
For who would bear the whips and scorns of time,\
Th'oppressor's wrong, the proud man's contumely,\
The pangs of dispriz'd love, the law's delay,\
The insolence of office, and the spurns\
That patient merit of th'unworthy takes,\
When he himself might his quietus make\
With a bare bodkin? Who would fardels bear,\
To grunt and sweat under a weary life,\
But that the dread of something after death,\
The undiscovere'd country, from whose bourn\
No traveller returns, puzzles the will,\
And makes us rather bear those ills we have\
Than fly to others that we know not of?\
Thus conscience does make cowards of us all,\
And thus the native hue of resolution\
Is sicklied o'er with the pale cast of thought,\
And enterprises of great pitch and moment\
With this regard their currents turn awry\
And lose the name of action."

with open('poem.txt','w') as b:
  b.write(a)

In [ ]:
#load text
raw_text1 = load_doc('poem.txt')
print(raw_text1)

# clean
tokens1 = raw_text1.split()
raw_text1 = ' '.join(tokens1)

# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text1)):
    # select sequence of tokens
    seq = raw_text1[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

To be, or not to be, that is the question:Whether tis nobler in the mind to sufferThe slings and arrows of outrageous fortune,Or to take arms against a sea of troublesAnd by opposing end them. To die—to sleep,No more; and by a sleep to say we endThe heart-ache and the thousand natural shocksThat flesh is heir to: 'tis a consummationDevoutly to be wish'd. To die, to sleep;To sleep, perchance to dream—ay, there's the rub:For in that sleep of death what dreams may come,When we have shuffled off this mortal coil,Must give us pause—there's the respectThat makes calamity of so long life.For who would bear the whips and scorns of time,Th'oppressor's wrong, the proud man's contumely,The pangs of dispriz'd love, the law's delay,The insolence of office, and the spurnsThat patient merit of th'unworthy takes,When he himself might his quietus makeWith a bare bodkin? Who would fardels bear,To grunt and sweat under a weary life,But that the dread of something after death,The undiscovere'd country, fr

In [ ]:
# save sequences to file
out_filename1 = 'char_sequences1.txt'
save_doc(sequences, out_filename1)

In [ ]:
in_filename = 'char_sequences1.txt'
raw_text1 = load_doc(in_filename)
lines = raw_text1.split('\n')

In [ ]:
# integer encode sequences of characters
chars = sorted(list(set(raw_text1)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append(encoded_seq)

# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 44


In [ ]:
model = Sequential()
model.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history=model.fit(X, y, validation_split = 0.3, epochs=50)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 200)               196000    
_________________________________________________________________
dropout_7 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 44)                8844      
Total params: 204,844
Trainable params: 204,844
Non-trainable params: 0
_________________________________________________________________
None
Train on 954 samples, validate on 409 samples
Epoch 1/50
954/954 [==============================] - 3s 3ms/step - loss: 3.4536 - acc: 0.1300 - val_loss: 3.0893 - val_acc: 0.1002
Epoch 2/50
954/954 [==============================] - 1s 902us/step - loss: 3.1040 - acc: 0.1300 - val_loss: 3.0352 - val_acc: 0.1589
Epoch 3/50
954/954 [==============================] - 1s 953u

In [ ]:
# save the model to file
model.save('model1.h5')
# save the mapping
dump(mapping, open('mapping1.pkl', 'wb'))

In [ ]:
# load the model
model = load_model('model1.h5')
# load the mapping
mapping = load(open('mapping1.pkl', 'rb'))

In [ ]:
# test start of rhyme
print(generate_seq(model, mapping, 10, 'To be or', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'Must give us', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'hello worl', 20))

To be or nnd the s ordsThang
Must give us pause—there's the r
hello worllfffi,, therhe's to 


In [ ]:
"summary below"

'summary below'

I tried using the famous Hamlet poem for this. It is much longer so took longer to train. So I decreased the number of epochs. That also increased the accuracy of the validation test and was also much faster to complete. The original 80 epochs made the train accuracy all the way to 1 so was not practical. 